In [1]:
from pygalfitm import PyGalfitm
from pygalfitm.VOs import splus
from pygalfitm.auxiliars import string_times_x, get_dims, get_exptime, unpack_file
from pygalfitm.psf import make_psf

import splusdata

import os

In [2]:
name = "ari_test"

ra = 52.430082897775755	
dec = -33.557141932864596
cut_size = 200
bands = ["I", "R", "G"]

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [3]:
conn = splusdata.connect("gustavo", "asdflkjh")

You have access to internal data


In [4]:
pyg = PyGalfitm() 

In [5]:
import pandas as pd
from astropy.io.fits import getheader

input_images = ""
psf_images = ""
filters = ""
zps = ""

df = pd.read_csv("pygalfitm/VOs/splus_idr4_zps.csv")

header = getheader(os.path.join(DATA_FOLDER, name + "_r.fits"))
field = header['OBJECT']

for band in bands:
    try:
        conn.get_cut(ra, dec, 200, band, filepath=os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'))
    except Exception as e:
        print(e)
    
    try:
        unpack_file(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits.fz'))
    except Exception as e:
        print(e)
        print("Make sure you have fpack (cfitsio) in your system alias.")
    
    make_psf(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'), outfile=os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits'))

    input_images += "," + os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits')
    psf_images += "," + os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits')
    filters += "," + str(band).lower()

    zps += "," + str(df[df['Field'] == field][f'ZP_{band.lower().replace("f", "J0")}'].values[0])


input_images = input_images[1:]
psf_images = psf_images[1:]
filters = filters[1:]
zps = zps[1:]

axis_ratios, effective_rs, position_angles, mags = splus.get_sersic_splus(conn, ra, dec, bands)


pyg.activate_components()
pyg.activate_components(["sersic"])

pyg.set_base({
    "A": input_images,
    "A1": filters,
    "B": os.path.join(OUTPUT_FOLDER, name + "ss.fits"),
    "C": "none",
    "D": psf_images,
    "A2": "7625,6231,4770",
    "H": f"1   {cut_size}  1   {cut_size}",
    "I": f"{cut_size} {cut_size}",
    "J": zps,
    "K": "0.55 0.55"
})

pyg.set_component("sersic", {
    "1": ( string_times_x(cut_size / 2, 3), 1, "band" ),
    "2": ( string_times_x(cut_size / 2, 3), 1, "band" ), 
    "3": ( mags, 3, "band" ), 
    "4": ( effective_rs, 2, "band" ),
    "5": ( string_times_x("4", 3), 2, "band" ),
    "9": ( axis_ratios, 1, "band" ),
    "10": ( position_angles, 1, "band" )
})

pyg.write_feedme()


File data/ari_test_i.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_r.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
File data/ari_test_g.fits.fz already exists. If you mean to replace it then use the argument "overwrite=True".
finished
finished
finished


In [9]:
_ = pyg.run()